In [1]:
path = "/Users/amrtamer/Documents/Internship/"

In [2]:
import sys
sys.path.append(path + 'Models')
sys.path.append(path + 'Models/Universal/uni2ts/myenv/lib/python3.10/site-packages/')
import utils as ut
import models

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
# os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import optuna
import torch.nn as nn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [4]:
data = pd.read_csv(path + r'data/Volve/Volve_cleaned_prepared.csv')
if "Unnamed: 0" in data.columns:
    data = data.drop(columns="Unnamed: 0")
data.head()

,well_name,WELL_BORE_CODE,date,prod_hrs,bhp,bht,dp_tubing,AVG_ANNULUS_PRESS,AVG_CHOKE_SIZE_P,whp,...,linear_regressor,EMA_short,EMA_medium,EMA_long,Rolling_short,Rolling_medium,Rolling_long,Lag_short,Lag_medium,Lag_long
0,7405,NO 15/9-F-1 C,2014-04-22,0.268405,1.029311,0.605973,0.396167,-2.016027,-0.533280,3.281836,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,7405,NO 15/9-F-1 C,2014-04-23,0.268405,0.845795,0.634777,0.239818,0.088113,-0.419656,2.849981,...,-0.101892,-0.103579,-0.104768,-0.106929,-0.001355,-0.002618,-0.00519,"[0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, -0.10230979661544379]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7405,NO 15/9-F-1 C,2014-04-24,0.268405,0.775025,0.639809,0.199890,-0.688434,-0.402874,2.607708,...,-0.101892,-0.009618,-0.053295,-0.079801,-0.001355,-0.002618,-0.00519,"[0.0, -0.10230979661544379, -0.10230979661544379]","[0.0, 0.0, 0.0, -0.10230979661544379, -0.10230...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.10..."
3,7405,NO 15/9-F-1 C,2014-04-25,0.268405,0.705033,0.642084,0.161493,1.145283,-0.379061,2.364038,...,-0.114759,-0.020781,-0.051611,-0.076565,-0.001355,-0.002618,-0.00519,"[-0.10230979661544379, -0.10230979661544379, 0...","[0.0, 0.0, -0.10230979661544379, -0.1023097966...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.1023097..."
4,7405,NO 15/9-F-1 C,2014-04-26,0.268405,0.625357,0.643889,0.117221,1.145283,-0.340115,2.088747,...,0.082297,-0.004428,-0.037199,-0.066767,-0.009496,-0.002618,-0.00519,"[-0.10230979661544379, 0.1777063666263608, -0....","[0.0, -0.10230979661544379, -0.102309796615443...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.102309796615..."


In [5]:
well_info = dict(set([(j, i) for i, j in data[['WELL_BORE_CODE', 'well_name']].values.tolist()])) # creating a dictionary of well identiification with {code: name} structure
well_info

{5769: 'NO 15/9-F-5 AH',
 7289: 'NO 15/9-F-15 D',
 5599: 'NO 15/9-F-12 H',
 7078: 'NO 15/9-F-11 H',
 7405: 'NO 15/9-F-1 C',
 5351: 'NO 15/9-F-14 H'}

In [6]:
univariate_feature = "oil_rate"
days_window = 5
predictions_days = 14
training_gap = days_window + predictions_days

In [7]:
# sequences = []
# for well_name in data['well_name'].unique():
#     well_data = data[data['well_name'] == well_name].sort_values(by='date').reset_index()
#     for i in range(len(well_data) - training_gap + 1):
#         inputs = well_data.iloc[i:i + days_window][univariate_feature].values
#         targets = well_data.iloc[i + days_window:i + days_window + predictions_days][univariate_feature].values
#         inputs_dates = list(well_data.iloc[i:i + days_window]['date'])
#         targets_dates = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['date'])
#         inputs_index = list(well_data.iloc[i:i + days_window]['index'])
#         targets_index = list(well_data.iloc[i + days_window:i + days_window + predictions_days]['index'])
#         sequences.append((well_name, inputs_dates, targets_dates, inputs_index, targets_index, inputs, targets))
#         if len(well_data.iloc[i:i + days_window + predictions_days]['well_name'].unique()) > 1:
#           print('violation')

# dataset = pd.DataFrame(sequences, columns=['well_name', 'inputs_dates', 'targets_dates', 'inputs_index', 'targets_index', 'inputs', 'targets', ])
# dataset.to_pickle(path + "/data/Volve/all_data_univariate.pkl")
# dataset

In [8]:
train_size = 0.6
val_size = 0.2
test_size = 0.2

In [9]:
test_size_param1 = test_size
test_size_param2 = val_size / (1 - test_size_param1)

In [10]:

# train_wells = []
# test_wells = []
# for i in dataset['well_name'].unique():
#     well = dataset[dataset['well_name'] == i]
#     train_set_length = int((train_size + val_size) * len(well))
#     train_wells.append(well[:train_set_length + 1])
#     test_set_length = int(test_size * len(well))
#     test_wells.append(well[-test_set_length:])

# # Flatten the lists of DataFrames
# temp_data = pd.concat(train_wells).reset_index(drop=True)
# data_test = pd.concat(test_wells).reset_index(drop=True)

# data_train, data_val = train_test_split(temp_data, test_size=test_size_param2, stratify=temp_data['well_name'], shuffle=True)

# data_train.reset_index(drop=True, inplace=True)
# data_val.reset_index(drop=True, inplace=True)
# data_test.reset_index(drop=True, inplace=True)

# data_train.to_pickle(path + "/data/Volve/data_univariate_train.pkl")
# data_val.to_pickle(path + "/data/Volve/data_univariate_val.pkl")
# data_test.to_pickle(path + "/data/Volve/data_univariate_test.pkl")

In [11]:
all_data = pd.read_pickle(path + "data/Volve/all_data_univariate.pkl")
data_train = pd.read_pickle(path + "data/Volve/data_univariate_train.pkl")
data_val = pd.read_pickle(path + "data/Volve/data_univariate_val.pkl")
data_test = pd.read_pickle(path + "data/Volve/data_univariate_test.pkl")
data_train.shape, data_val.shape, data_test.shape

((4730, 7), (1577, 7), (1573, 7))

In [12]:
batch_size = 50

In [13]:
train_dataset = ut.UivariateTimeSeriesDataset(data_train)
val_dataset = ut.UivariateTimeSeriesDataset(data_val)
test_dataset = ut.UivariateTimeSeriesDataset(data_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [14]:
visualization_test_dataset = ut.UnivariateVisualizationTimeSeriesDataset(data_test)

visualization_test_loader = DataLoader(visualization_test_dataset, batch_size=int(len(test_dataset) / 50), shuffle=True)

In [15]:
size = "small"
num_samples = 50

In [16]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    num_samples = trial.suggest_int("num_samples", 10, 50, step=10)
    size = trial.suggest_categorical("size", ["tiny", "mini", "small", "base", "large"])
    # Load the model
    pipeline = ChronosPipeline.from_pretrained(
        f"amazon/chronos-t5-{size}",
        device_map="mps",  # use "cpu" for CPU inference and "mps" for Apple Silicon
        torch_dtype=torch.bfloat16,  # torch.bfloat16,
    )
    
    avg_loss = 0
    
    for X_batch, Y_batch in val_loader:
        
        forecast = pipeline.predict(
            context=X_batch,
            prediction_length=predictions_days,
            num_samples=num_samples,
        )
        
        prediction = forecast.mean(axis=1)
        prediction = prediction.view(prediction.shape[0] * prediction.shape[1])
        Y_batch = Y_batch.view(Y_batch.shape[0] * Y_batch.shape[1])
        
        loss = F.mse_loss(prediction, Y_batch)
        avg_loss += loss.item()
    
    val_loss = avg_loss / len(val_loader)
    return val_loss

In [14]:
# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

[I 2024-07-08 14:02:15,492] A new study created in memory with name: no-name-561035a0-8901-401f-926f-1e5fa9fdb5e0
/Users/amrtamer/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[I 2024-07-08 14:03:14,934] Trial 0 finished with value: 0.07399260968668386 and parameters: {'num_samples': 50, 'size': 'small'}. Best is trial 0 with value: 0.07399260968668386.
[I 2024-07-08 14:03:50,849] Trial 1 finished with value: 0.08172283679596148 and parameters: {'num_samples': 20, 'size': 'small'}. Best is trial 0 with value: 0.07399260968668386.
[I 2024-07-08 14:04:20,112] Trial 2 finished with value: 0.06905774885672145 and parameters: {'num_samples': 30, 'size': 'mini'}. Best is trial 2 with value: 0.06905774885672145.
[I 2024-07-08 14:09:04,553] Trial 3 finishe

Best parameters: {'num_samples': 50, 'size': 'tiny'}


In [16]:
size = "small"
num_samples = 50

In [17]:
model = models.Moirai(size=size, input_length=days_window, output_length=predictions_days, num_samples=num_samples)

In [19]:
train_metrics = ut.evaluate(model, train_loader)
val_metrics = ut.evaluate(model, val_loader)
test_metrics = ut.evaluate(model, test_loader)

Metrics on dataset:

Loss is 68234351.7811
MAE is 125.3311
RMSE is 8273.4922
R² is -63142952.0000


Metrics on dataset:

Loss is 4996661576.4665
MAE is 736.6165
RMSE is 67147.2969
R² is -4066785280.0000


Metrics on dataset:

Loss is 0.0026
MAE is 0.0335
RMSE is 0.0509
R² is 0.8916




In [18]:
ut.produce_graphs(model, visualization_test_loader, data, data_test, days_window, predictions_days, path + "Models/Universal/Moirai/Graphs", well_info, test_size)
test_metrics = ut.evaluate(model, test_loader)

Metrics on dataset:

Loss is 0.0026
MAE is 0.0334
RMSE is 0.0509
R² is 0.8918


